# Models

> Fill in a module description here


In [ ]:
#| default_exp vision.models

In [ ]:
#| hide
from nbdev.showdoc import *  # noqa: F403

In [ ]:
#| export
from fastcore.utils import *  # noqa: F403
from torch import nn
import torch.nn.functional as F
import torch
from peft import *  # noqa: F403

## MNIST models

In [ ]:
# class MNISTCNNEncoder(nn.Module):
#     def __init__(self, in_channels=1, args=None):
#         super(MNISTCNNEncoder, self).__init__()
#         self.conv1 = nn.Conv2d(in_channels, 10, kernel_size=5)
#         self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
#         self.conv2_drop = nn.Dropout2d()
#         self.fc1 = nn.Linear(320, 50)
#         # self.fc2 = nn.Linear(50, 10)

#     def forward(self, x):
#         x = F.relu(F.max_pool2d(self.conv1(x), 2))
#         x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
#         x = x.view(-1, x.shape[1]*x.shape[2]*x.shape[3])
#         x = F.relu(self.fc1(x))
#         return x
        # x = F.dropout(x, training=self.training)
        # x = self.fc2(x)

In [ ]:
#| export
class MNISTCNNEncoder(nn.Module):
    def __init__(self, in_channels=1, img_size=28, hidden_dim=512):
        super(MNISTCNNEncoder, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, 32, kernel_size=5, stride=1, padding=2)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=5, stride=1, padding=2)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)

        final_size = img_size // 2 // 2
        self.fc1 = nn.Linear(64 * final_size * final_size, hidden_dim)
        self.relu = nn.ReLU()

    def forward(self, x):
        if x.ndim == 4 and x.shape[1] not in (1, 3) and x.shape[-1] in (1, 3):
            x = x.permute(0, 3, 1, 2)
        x = self.relu(self.conv1(x))
        x = self.pool(x)
        x = self.relu(self.conv2(x))
        x = self.pool(x)
        x = x.reshape(x.size(0), -1)# x.view(x.size(0), -1)  # Flatten
        x = self.fc1(x)
        return x
    

# Classification Head Model
class MNISTCNNClassificationHead(nn.Module):
    def __init__(self, hidden_dim= 512, num_classes=10):
        super(MNISTCNNClassificationHead, self).__init__()
        self.fc2 = nn.Linear(hidden_dim, num_classes)
        self.dropout = nn.Dropout(0.5) 

    def forward(self, x):
        x = self.dropout(x)
        x = self.fc2(x)
        return x
    


class MNISTCNN(nn.Module):
    def __init__(self, in_channels=1, img_size= 28, hidden_dim=512, num_classes=10):
        super(MNISTCNN, self).__init__()
        self.in_channels = in_channels
        self.encoder = MNISTCNNEncoder(in_channels=in_channels, img_size= img_size, hidden_dim=hidden_dim)
        self.classifier = MNISTCNNClassificationHead(hidden_dim= hidden_dim, num_classes= num_classes)

    def forward(self, x):
        if x.ndim == 4 and x.shape[1] not in (1, 3) and x.shape[-1] in (1, 3):
            x = x.permute(0, 3, 1, 2)
        x = self.encoder(x)
        x = self.classifier(x)
        return x
    

In [ ]:
#| hide
inp = torch.randn(1, 3, 32, 32)  # Example input
enc = MNISTCNNEncoder(in_channels=3, img_size=32, hidden_dim=512)

print(enc(inp).shape)
model = MNISTCNN(in_channels=3, img_size=32, hidden_dim=512, num_classes=10)
out = model(inp)
print(out.shape)  # Should print torch.Size([1, 10])

torch.Size([1, 512])
torch.Size([1, 10])


## CIFAR10 models

In [ ]:
#| export
import torch
import torch.nn as nn
import numpy as np
import random

torch.manual_seed(42)  
np.random.seed(42)
random.seed(42)

# Encoder Model
class CIFAR10Encoder(nn.Module):
    def __init__(self, in_channels=3, img_size= 32, hidden_dim=512):
        super(CIFAR10Encoder, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, 32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.relu = nn.ReLU()

        # Compute the flattened size dynamically at initialization
        final_size = img_size // 2 // 2 // 2  # 3 pool layers divide the size by 2 three times
        self.fc1 = nn.Linear(128 * final_size * final_size, hidden_dim)


    def forward(self, x):
        x = self.pool(self.relu(self.conv1(x)))
        x = self.pool(self.relu(self.conv2(x)))
        x = self.pool(self.relu(self.conv3(x)))
        x = x.view(x.size(0), -1)  # Flatten
        x = self.relu(self.fc1(x))
        return x

# Classification Head Model
class CIFAR10ClassificationHead(nn.Module):
    def __init__(self, hidden_dim= 512, num_classes=10):
        super(CIFAR10ClassificationHead, self).__init__()
        self.fc2 = nn.Linear(hidden_dim, num_classes)
        self.dropout = nn.Dropout(0.5) 

    def forward(self, x):
        x = self.dropout(x)
        x = self.fc2(x)
        return x

class CIFAR10Model(nn.Module):
    def __init__(self, in_channels=3, img_size= 32, hidden_dim=512, num_classes=10):
        super(CIFAR10Model, self).__init__()
        self.encoder = CIFAR10Encoder(in_channels=in_channels, img_size= img_size, hidden_dim=hidden_dim)
        self.classifier = CIFAR10ClassificationHead(hidden_dim= hidden_dim, num_classes= num_classes)

    def forward(self, x):
        x = self.encoder(x)
        x = self.classifier(x)
        return x


In [ ]:
#| hide
model = CIFAR10Model(in_channels=3, img_size= 32,hidden_dim=1024, num_classes=100)
inp = torch.randn(1, 3, 32, 32)
out = model(inp)
print(out.shape)  # Should output torch.Size([1, 512])


torch.Size([1, 100])


In [ ]:
# from torchtune.models.llama2 import llama2_7b, lora_llama2_7b

In [ ]:
# base_model = llama2_7b()

: 

In [ ]:
# lora_model = lora_llama2_7b(lora_attn_modules=["q_proj", "v_proj"])

In [ ]:
# import os
# import urllib.request
# import json 
# # download th e dataset from link
# file_path = "https://huggingface.co/datasets/databricks/databricks-dolly-15k/resolve/main/databricks-dolly-15k.jsonl"

# # get the file name from the url
# file_name = os.path.basename(file_path)
# # download the file
# urllib.request.urlretrieve(file_path, file_name)
# # read the file
# with open(file_name, 'r') as f:
#     data = [json.loads(line) for line in f]
# # print the first 5 lines
# print(data[:5])

[{'instruction': 'When did Virgin Australia start operating?', 'context': "Virgin Australia, the trading name of Virgin Australia Airlines Pty Ltd, is an Australian-based airline. It is the largest airline by fleet size to use the Virgin brand. It commenced services on 31 August 2000 as Virgin Blue, with two aircraft on a single route. It suddenly found itself as a major airline in Australia's domestic market after the collapse of Ansett Australia in September 2001. The airline has since grown to directly serve 32 cities in Australia, from hubs in Brisbane, Melbourne and Sydney.", 'response': 'Virgin Australia commenced services on 31 August 2000 as Virgin Blue, with two aircraft on a single route.', 'category': 'closed_qa'}, {'instruction': 'Which is a species of fish? Tope or Rope', 'context': '', 'response': 'Tope', 'category': 'classification'}, {'instruction': 'Why can camels survive for long without water?', 'context': '', 'response': 'Camels use the fat in their humps to keep th

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()